In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210801.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,28/07/2021 08:41:04,Mommie Dearest (1981),The Celebration (1998),"Honey, I Shrunk the Kids (1989)",MEXICO.,[Default to Nicolas Cage],IN THE DESERT,CUMMING OF AGE
1,28/07/2021 08:45:22,The Celebration (1998),Mommie Dearest (1981),NaN,CUMMING OF AGE,IN THE DESERT,MEXICO.,[Default to Nicolas Cage]
2,28/07/2021 10:58:52,"Honey, I Shrunk the Kids (1989)",The Celebration (1998),Mommie Dearest (1981),MEXICO.,IN THE DESERT,[Default to Nicolas Cage],CUMMING OF AGE
3,29/07/2021 15:26:29,"Honey, I Shrunk the Kids (1989)",Mommie Dearest (1981),The Celebration (1998),CUMMING OF AGE,MEXICO.,IN THE DESERT,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Mommie Dearest (1981)', 'The Celebration (1998)',
       'Honey, I Shrunk the Kids (1989)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Mommie Dearest (1981)': <Candidate('Mommie Dearest (1981)')>,
 'The Celebration (1998)': <Candidate('The Celebration (1998)')>,
 'Honey, I Shrunk the Kids (1989)': <Candidate('Honey, I Shrunk the Kids (1989)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Mommie Dearest (1981), The Celebration (1998), Honey, I Shrunk the Kids (1989))>,
 <Ballot(The Celebration (1998), Mommie Dearest (1981))>,
 <Ballot(Honey, I Shrunk the Kids (1989), The Celebration (1998), Mommie Dearest (1981))>,
 <Ballot(Honey, I Shrunk the Kids (1989), Mommie Dearest (1981), The Celebration (1998))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
Honey, I Shrunk the Kids (1989)        2  Elected
The Celebration (1998)                 1  Rejected
Mommie Dearest (1981)                  1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
MEXICO.                          2  Elected
CUMMING OF AGE                   2  Rejected
IN THE DESERT                    0  Rejected
[Default to Nicolas Cage]        0  Rejected

